<div align="center>
            <h1>
            TUTORIAL
            </h1>
</div>    

<h1>
    <center>
        -- TUTORIAL --
    </center>
</h1>


<br>
<br>
    


In this notebook, we are going to have a good tour of main features, usages and how-to of the package.

<br>

## 1. Install
****************

As requested we need to install the package from pypi. Depends of your internet connection, it could take 1 or 2 minutes.

In [1]:
!pip install legal_doc_processing==2.2.3


<br>
<br>

## 2. package integrity checks and third party downloads
************

<br>

As we will use major NLP librairies, we need to download data collections and mandory web assets such as NLTK stop words, scapy or transformers models or tensorflow/pyTorch assets. It can take 1/2 minutes (depends of your web connection).

In [3]:
from legal_doc_processing import boot
boot.boot()

PressRelease(source:cftc, {'cooperation_credit': '', 'court': 'U.S. Dis', 'currency': 'USD', 'decision_date': '2015-01-', 'extracted_sanctions': 'none of ', 'extracted_violations': 'Defendan', 'folder': '-- DUMMY', 'judge': 'Marcia M', 'monitor': '0', 'nature_de_sanction': 'none of ', 'nature_of_violations': '', 'reference': '-- DUMMY', 'extracted_authorities': 'U.S. Dis', 'type': 'Order', 'justice_type': 'U.S. - C', 'defendant': 'Chawalit', 'country_of_violation': 'United S', 'penalty_details': '', 'monetary_sanction': '1000000', 'compliance_obligations': ''}, pipe/spacy:OK/OK

<br>
<br>

## 3. Understand bascic package structure
********************
<br>

There is 3 main modules in legal-doc-processing : 
- legal_doc for LegalDoc objects ie order, complaint, etc etc all kinds of official documents
- press_release for PressRelease objects for legal press release related to each case
- decision for both LegalDoc and PressRelease documents. The Decision object is able to read both legal document and press release, make prediction from both documents and merge/clean predictions of both documents.


so you can :

In [ ]:
from legal_doc_processing import legal_doc # import legal document module
from legal_doc_processing import press_release # import press release module
from legal_doc_processing import decision # import decision module


each package has its own Object : 

In [ ]:
print(f"press_release.PressRelease : {press_release.PressRelease} \n")
print(f"press_release.__doc__ : {press_release.__doc__} \n")
print(f"press_release.__dict__ : {press_release.__dict__}")


as consequence it is recommanded to use the import method : 

In [ ]:
from legal_doc_processing import press_release as pr

print(pr.PressRelease)

LegalDoc, PressRelease and Decision are inheritance of The Base class. You can consider The Base class is an
as an abstraction of 3 others objecst.

Of course you can call this class, even if you will not mainuplate it 

In [ ]:
from legal_doc_processing.base.base import Base

print(Base)

<br>
<br>

## 4 Object instanciation
*****************
<br>

youn can init an object in 3 ways.: 
- given a string
- given a text file path
- give an url

this three methods are onboarded feature form the package: 



### 4.1 with a string

In [ ]:
pr.PressRelease
# or 
pr.from_text

print(pr.PressRelease)
print(pr.from_text)

### 4.2 with a file

In [ ]:
pr.from_file
print(pr.from_file)

### 4.3 with an url

In [ ]:
pr.from_url
print(pr.from_url)

We are going to initate  an object directly form web.
First we need an url, and then we need the source

In [ ]:
url = "https://storage.googleapis.com/theolex_documents_processing/cftc/text/7100-15/press-release.txt"
source = "cftc"

then we can instantiate our object 

In [ ]:
press = pr.from_url(url=url, source=source)

print(press)

as we can see, the object take between 0.3 and 1 seconds to instatialte.
it is due to NLP complex methods to preporcess and restructure the document.

to so for each object, we need to initiate complex object such as spacy instance, transformers instance etc etc.

in our case, jsut for one object it is not a problem but suposed we have 1000 document to handle ? 


There is of course a solution : instaicate spacy and transformers object outside our PressRelease object.

### 4.4 instanciate 100 objects

In [ ]:
from legal_doc_processing.utils import get_pipeline, get_spacy

%time nlspa = get_spacy()
%time nlpipe=get_pipeline()

as we can see  nlpipe needs more than 1 sec to instanciate

now we can easy loop on 100 objects in a very fast way 

In [ ]:
import requests
txt = requests.get(url).text
txt_list = [txt for i in range(10)]

make_press = lambda i : pr.PressRelease(i, source=source, nlspa=nlspa, nlpipe = nlpipe)
%time press_list = [make_press(i) for i in txt_list]


### 4.4 main attributes

lets have a tour of this objects

In [ ]:
press = press_list[0]
print(f"press : {press} \n")

print(f"press.__doc__ : {press.__doc__ } \n" )

print(f"press.__dict__.keys() : {press.__dict__.keys() } \n" )

we can find some basics attributes such as : 

In [ ]:
basic_attrs = ['obj_name', 'juridiction', 'source', 'raw_text', ]
    
strize = lambda attr :  str(getattr(press, attr))[:200].replace('\n', '/n')
for attr in  basic_attrs : 
    print(f"{attr.ljust(20)} : {strize(attr)}")
          

we can find some specific attrs relative to the document structure

In [ ]:
basic_attrs = ['date', 'h1', 'header', 'content', 'struct_content', 'abstract', 'end']
    
strize = lambda attr :  str(getattr(press, attr))[:200].replace('\n', '/n')
for attr in  basic_attrs : 
    print(f"{attr.ljust(20)} : {strize(attr)}")
          

and even more interessant we can find a list of data points in : 

In [ ]:
import pprint 

pprint.pprint(press.feature_dict)


<br>
<br>

## 5. Predictions
**********************

<br>

### 5.1 specific predictions


you can now make predictions : 

In [ ]:
%time defendant =  press.predict("defendant")

In [ ]:
we have to ways of getting a predictions

<br>

### 5.2 raw vs final predicstions 

fist one is the result of the prediction wich is a list of tupples with answer, score suct as 

In [ ]:
print(defendants)

you can acces to the same object with the spécial _feature_dict attribute and underscored feature

In [ ]:
print(press._feature_dict["_defendant"])

or if you want a more usable object , you acces to the str readble answer with the feature_dict argument

In [ ]:
print(press.feature_dict["defendant"])

<br>

### 5.3 make all predictions at once

and of course you have a predic_all and predict("all") methods to make all predictions in once

In [ ]:
%time _ = press.predict_all()

In [ ]:
pprint.pprint(press.feature_dict)

or if you want the raw value of each feature

In [ ]:
pprint.pprint(press._feature_dict)